In [10]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import pickle

# Analyze a problem

In [3]:
# imports
from lib.problems import *
from lib.algorithms.path_formulation import PathFormulation

In [4]:
# define problem params
prob_name = 'delta'
num_paths = 4
edge_disjoint = True
dist_metric = 'inv-cap'
# create problem
prob = get_problem(prob_name, model='gravity', scale_factor=8)

In [13]:
# takes in a problem, returns a 4-tuple (id, src, dst, capacity) for each link
def get_edge_capacities(prob):
  capacities = []
  edge_id_mapping = dict()
  cur_edge_id = 0
  # get adjacency list for each node in graph
  adj_list = prob.G.adj
  for src, src_info in adj_list.items():
    for dst, dst_info in src_info.items():
      capacities.append((src, dst, dst_info['capacity']))
      key = (src, dst)
      val = (cur_edge_id, dst_info['capacity'])
      edge_id_mapping[key] = val
      cur_edge_id += 1
  
  # sort on src ID for consistent edge ordering
  capacities = sorted(capacities, key=lambda x: x[0])
  print(f"# edges: {len(capacities)}")
  return capacities, edge_id_mapping

# extract flows from problem
def get_flows(prob):
  flows = []
  for i, src, dst, flow_size in prob.multi_commodity_list:
    # src and dst are a 1-length list
    flows.append((src[0], dst[0], flow_size))
  print(f"# flows: {len(flows)}")
  return flows

# extract traffic matrix
def get_traffic_matrix(prob):
  traffic_mat = prob.traffic_matrix._tm
  print(f"# nodes: {traffic_mat.shape[0]}")
  return traffic_mat

# map paths to edge IDs, returns mapped paths
# edge_id_mapping: (src, dst) -> (edge_id, capacity)
# paths: (src, dst) -> [path1, path2, ...], path1 = [node1, node2, ...]
def map_paths_to_edge_ids(paths, edge_id_mapping):
  # path_id -> [edge_id1, edge_id2, ...]
  path_ids_to_edge_ids_mapping = dict()
  # (src, dst) -> [path_id1, path_id2, ...]
  new_paths = dict()
  cur_path_id = 0
  def get_edge_id(src, dst):
    val = edge_id_mapping.get((src, dst), None) or edge_id_mapping.get((dst, src), None)
    if val is None:
      raise Exception(f"edge ({src}, {dst}) not found in edge_id_mapping")
    return val[0]
  # iterate over all (src, dst) pairs
  for (src_id, dst_id), pair_paths in paths.items():
    new_paths[(src_id, dst_id)] = []
    # iterate over all paths for this (src, dst) pair
    for path_nodes in pair_paths:
      path_edges = [(path_nodes[i], path_nodes[i+1]) for i in range(len(path_nodes)-1)]
      path_edge_ids = [get_edge_id(*edge) for edge in path_edges]
      # add to mapping
      path_ids_to_edge_ids_mapping[cur_path_id] = np.asarray(path_edge_ids, dtype=np.uint32)
      new_paths[(src_id, dst_id)].append(cur_path_id)
      # increment path ID
      cur_path_id += 1
  return new_paths, path_ids_to_edge_ids_mapping
  

In [22]:
# extract paths
paths_dict = PathFormulation.read_paths_from_disk_or_compute(prob, num_paths, edge_disjoint, dist_metric)
print(f"Avg. path length: {np.mean([len(p)-1 for p in paths_dict.values()]):.2f}")
# extract flows from problem
prob_flows = get_flows(prob)
# extract edge capacities from problem
edge_caps, edge_id_mapping = get_edge_capacities(prob)
# process paths dict into list of edge IDs per (flow ID, path ID) pair
mapped_paths, path_ids_to_edge_ids_mapping = map_paths_to_edge_ids(paths_dict, edge_id_mapping)
# extract traffic matrix
prob_tm = get_traffic_matrix(prob)

Loading paths from pickle file /home/sauce/POP/traffic_engineering/topologies/paths/path-form/Deltacom.graphml-4-paths_edge-disjoint-True_dist-metric-inv-cap-dict.pkl
paths_dict size: 12656
Avg. path length: 1.09
# flows: 12656
# edges: 322
# nodes: 113


# Formulate optimization problem
### Objective: max total flow

In [9]:
# create mapping of (flow_id, path) -> index
pair_idx_mapping = {}